# Project 1

In [2]:
#Importing pandas as pd
import pandas as pd

#Setting options to display full data frame rows
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
#Reading in bikeDetails data set using pandas
bike_data = pd.read_csv("bikeDetails.csv")

In [4]:
#Displays the first 5 observations from the bike_data data set
bike_data.head()

,name,selling_price,year,seller_type,owner,km_driven,ex_showroom_price
0,Royal Enfield Classic 350,175000,2019,Individual,1st owner,350,NaN
1,Honda Dio,45000,2017,Individual,1st owner,5650,NaN
2,Royal Enfield Classic Gunmetal Grey,150000,2018,Individual,1st owner,12000,148114.0
3,Yamaha Fazer FI V 2.0 [2016-2018],65000,2015,Individual,1st owner,23000,89643.0
4,Yamaha SZ [2013-2014],20000,2011,Individual,2nd owner,21000,NaN


## Ashley's EDA

## George's EDA

In [5]:
#using the .describe() method to get a first look at the numeric variables in the dataset. Although you would not necessarily perform math on the listed years for each bike, seeing info like the average year is meaningful.
bike_data.describe()

,selling_price,year,km_driven,ex_showroom_price
count,1061.000000,1061.000000,1061.000000,6.260000e+02
mean,59638.151744,2013.867107,34359.833176,8.795871e+04
std,56304.291973,4.301191,51623.152702,7.749659e+04
min,5000.000000,1988.000000,350.000000,3.049000e+04
25%,28000.000000,2011.000000,13500.000000,5.485200e+04
50%,45000.000000,2015.000000,25000.000000,7.275250e+04
75%,70000.000000,2017.000000,43000.000000,8.703150e+04
max,760000.000000,2020.000000,880000.000000,1.278000e+06


In [7]:
bike_data.columns
#note: our columns are name, selling_price, year, seller_type, owner, km_driven, and ex_showroom_price

Index(['name', 'selling_price', 'year', 'seller_type', 'owner', 'km_driven',
       'ex_showroom_price'],
      dtype='object')

In [11]:
bike_data_numeric = bike_data["selling_price", "km_driven", "ex_showrooom_price"]

KeyError: ('selling_price', 'km_driven', 'ex_showrooom_price')